# Linea de Muerte Z

##  1. Big Picture

## Inicializacion

In [1]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

[1] "2025-12-03 23:09:19 UTC"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,658083,35.2,1454578,77.7,1392875,74.4
Vcells,1221916,9.4,8388608,64.0,1975158,15.1


In [2]:
library(readr)
install.packages("data.table")   # si no lo tenías
library(data.table)

Installing package into ‘/home/juanjoselopez840/.local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [3]:
install.packages("dplyr")

Installing package into ‘/home/juanjoselopez840/.local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [4]:
library(data.table)

dataset2 <- fread(
  "/home/juanjoselopez840/Downloads/competencia_02_crudo.csv.gz"
)


In [5]:
library(data.table)

dataset3 <- fread(
  "/home/juanjoselopez840/Downloads/TERCERA/competencia_03_crudo.csv.gz"
)


In [6]:
library(dplyr)

dataset <- bind_rows(dataset2, dataset3)



Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [7]:
ncol(dataset)
nrow(dataset)

[1] 154

[1] 4883051

## Preprocesamiento

### Generacion de la clase_ternaria

### Eliminacion de Features

La auténtica  **salsa mágica**  de este script es la eliminación de estos dos atributos del dataset ya que tran problemas con el Data Drifting
* mprestamos_personales
* cprestamos_personales

el problema con estos campos se detectó manualmente mediante un analisis exploratorio de datos y análisis de corridas de LightGBM en meses previos.

In [8]:

# calculo el periodo0 consecutivo
dsimple <- dataset[, list(
  "pos" = .I,
  numero_de_cliente,
  periodo0 = as.integer(foto_mes/100)*12 +  foto_mes%%100 )
]


# ordeno
setorder( dsimple, numero_de_cliente, periodo0 )

# calculo topes
periodo_ultimo <- dsimple[, max(periodo0) ]
periodo_anteultimo <- periodo_ultimo - 1


# calculo los leads de orden 1 y 2
dsimple[, c("periodo1", "periodo2") :=
  shift(periodo0, n=1:2, fill=NA, type="lead"),  numero_de_cliente
]

# assign most common class values = "CONTINUA"
dsimple[ periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA" ]

# calculo BAJA+1
dsimple[ periodo0 < periodo_ultimo &
  ( is.na(periodo1) | periodo0 + 1 < periodo1 ),
  clase_ternaria := "BAJA+1"
]

# calculo BAJA+2
dsimple[ periodo0 < periodo_anteultimo & (periodo0+1 == periodo1 )
  & ( is.na(periodo2) | periodo0 + 2 < periodo2 ),
  clase_ternaria := "BAJA+2"
]

# pego el resultado en el dataset original y grabo
setorder( dsimple, pos )
dataset[, clase_ternaria := dsimple$clase_ternaria ]

rm(dsimple)
gc()
Sys.time()

Warning message in `[.data.table`(dataset, , `:=`(clase_ternaria, dsimple$clase_ternaria)):
“A shallow copy of this data.table was taken so that := can add or remove 1 columns by reference. At an earlier point, this data.table was copied by R (or was created manually using structure() or similar). Avoid names<- and attr<- which in R currently (and oddly) may copy the whole data.table. Use set* syntax instead to avoid copying: ?set, ?setnames and ?setattr. It's also not unusual for data.table-agnostic packages to produce tables affected by this issue. If this message doesn't help, please report your use case to the data.table issue tracker so the root cause can be fixed or this message improved.”


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,844566,45.2,1454578,77.7,1454578,77.7
Vcells,1301535789,9930.0,2146849683,16379.2,2004410542,15292.5


[1] "2025-12-03 23:10:37 UTC"

In [9]:
setorder( dataset, foto_mes, clase_ternaria, numero_de_cliente)
dataset[, .N, list(foto_mes, clase_ternaria)]

foto_mes,clase_ternaria,N
<int>,<chr>,<int>
201901,BAJA+1,645
201901,BAJA+2,729
201901,CONTINUA,122899
201902,BAJA+1,733
201902,BAJA+2,707
201902,CONTINUA,123961
201903,BAJA+1,708
201903,BAJA+2,751
201903,CONTINUA,124508


### ACTUALIZO

In [10]:
library(data.table)
datos <- list(
  foto_mes = c(
    201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908,
    201909, 201910, 201911, 201912, 202001, 202002, 202003, 202004,
    202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012,
    202101, 202102, 202103, 202104, 202105, 202106, 202107, 202108,
    202109
  ),
  
  indice = c(
    178.7447, 168.5467, 156.5534, 148.0056, 140.6648, 134.3681,
    129.2842, 120.576, 108.3169, 101.6406, 93.44802, 86.48553,
    82.36715, 78.72844, 72.93848, 70.42889, 67.83106, 64.13043,
    61.02986, 56.7784, 52.48125, 46.928, 42.45283, 36.95258,
    31.63138, 27.07382, 21.2434, 16.48667, 12.75869, 9.284533,
    6.103192, 3.546238, 0
  )
)

df <- as.data.table(datos)
df

foto_mes,indice
<dbl>,<dbl>
201901,178.744700
201902,168.546700
201903,156.553400
201904,148.005600
201905,140.664800
201906,134.368100
201907,129.284200
201908,120.576000
201909,108.316900


In [11]:
columnas_seleccionadas <- c(
  "foto_mes",
  "mrentabilidad", "mrentabilidad_annual", "mcomisiones", "mactivos_margen", "mpasivos_margen",
  "mcuenta_corriente_adicional", "mcuenta_corriente", "mcaja_ahorro", "mcaja_ahorro_adicional",
  "mcaja_ahorro_dolares", "mcuentas_saldo", "mautoservicio", "mtarjeta_visa_consumo",
  "mtarjeta_master_consumo", "mprestamos_prendarios",
  "mprestamos_hipotecarios", "mplazo_fijo_dolares", "mplazo_fijo_pesos", "minversion1_pesos",
  "minversion1_dolares", "minversion2", "mpayroll", "mpayroll2", "mcuenta_debitos_automaticos",
  "mttarjeta_visa_debitos_automaticos", "mttarjeta_master_debitos_automaticos", "mpagodeservicios",
  "mpagomiscuentas", "mcajeros_propios_descuentos", "mtarjeta_visa_descuentos",
  "mtarjeta_master_descuentos", "mcomisiones_mantenimiento", "mcomisiones_otras",
  "mforex_buy", "mforex_sell", "mtransferencias_recibidas", "mtransferencias_emitidas",
  "mextraccion_autoservicio", "mcheques_depositados", "mcheques_emitidos",
  "mcheques_depositados_rechazados", "mcheques_emitidos_rechazados", "matm", "matm_other",
  "Master_mfinanciacion_limite", "Master_msaldototal", "Master_msaldopesos", "Master_msaldodolares",
  "Master_mconsumospesos", "Master_mconsumosdolares", "Master_mlimitecompra",
  "Master_madelantopesos", "Master_madelantodolares", "Master_mpagado", "Master_mpagospesos",
  "Master_mpagosdolares", "Master_mconsumototal", "Master_mpagominimo",
  "Visa_mfinanciacion_limite", "Visa_msaldototal", "Visa_msaldopesos", "Visa_msaldodolares",
  "Visa_mconsumospesos", "Visa_mconsumosdolares", "Visa_mlimitecompra", "Visa_madelantopesos",
  "Visa_madelantodolares", "Visa_mpagado", "Visa_mpagospesos", "Visa_mpagosdolares",
  "Visa_mconsumototal", "Visa_mpagominimo"
)

In [12]:
subset_data <- dataset[, ..columnas_seleccionadas]

In [13]:
subset_data[, foto_mes := as.integer(foto_mes)]
df[, foto_mes := as.integer(foto_mes)]
df[, indice := as.numeric(indice)]

In [14]:
# Aseguro que son data.table
setDT(subset_data)
setDT(df)

# LEFT JOIN igual al merge de Python
subset <- merge(
  subset_data,
  df[, .(foto_mes, indice)],
  by = "foto_mes",
  all.x = TRUE
)

In [15]:
# Recorro columnas y actualizo todas menos foto_mes e indice
for (col in names(subset)) {
  if (!(col %in% c("foto_mes", "indice"))) {
    subset[, (col) := get(col) * (1 + indice / 100) ]
  }
}

cat("✅ Data actualizada correctamente a precios de julio 2021\n")
print(head(subset))


✅ Data actualizada correctamente a precios de julio 2021
Key: <foto_mes>
   foto_mes mrentabilidad mrentabilidad_annual mcomisiones mactivos_margen
      <int>         <num>                <num>       <num>           <num>
1:   201901      1294.602             86714.02   4174.3413     -1050.05916
2:   201901      2477.232             27281.61   2364.0059        59.73499
3:   201901      5476.915             51766.54   3377.1872      -169.08654
4:   201901      1848.189            122367.89   1810.0566       -64.33428
5:   201901      2041.833            100430.57    323.7062      -493.87986
6:   201901      3134.930             31462.50   1495.6326      1284.48345
   mpasivos_margen mcuenta_corriente_adicional mcuenta_corriente mcaja_ahorro
             <num>                       <num>             <num>        <num>
1:     -1404.95691                           0        -59106.977        0.000
2:        36.79430                           0        -13085.447        0.000
3:      1959.12

In [16]:
# Copio de subset hacia dataset en todas las columnas excepto foto_mes e indice
for (col in names(subset)) {
  if (!(col %in% c("foto_mes", "indice"))) {
    dataset[, (col) := subset[[col]] ]
  }
}

In [17]:
ncol(dataset)
nrow(dataset)

[1] 155

[1] 4883051

### IMPUTACION

In [18]:
# Imputo nulls en variables 0 de pandemia
# 201904 - ctarjeta_visa_debitos_automaticos, mttarjeta_visa_debitos_automaticos
dataset[foto_mes == 201904, c("ctarjeta_visa_debitos_automaticos", "mttarjeta_visa_debitos_automaticos") := NA]

# 201905 - mrentabilidad, mrentabilidad_annual, mcomisiones
dataset[foto_mes == 201905, c("mrentabilidad", "mrentabilidad_annual", "mcomisiones", "mactivos_margen", "mpasivos_margen", "ccomisiones_otras", "mcomisiones_otras") := NA]

# 201910 - mrentabilidad, mrentabilidad_annual, mcomisiones

dataset[foto_mes == 201910, c("mrentabilidad", "mrentabilidad_annual", "mcomisiones", "mactivos_margen", "mpasivos_margen", "ccajeros_propios_descuentos", "mcajeros_propios_descuentos", "ctarjeta_visa_descuentos", "mtarjeta_visa_descuentos", "ctarjeta_master_descuentos", "mtarjeta_master_descuentos", "ccomisiones_otras", "mcomisiones_otras", "chomebanking_transacciones") := NA]

# 202002 - ccajeros_propios_descuentos, mcajeros_propios_descuentos, ctarjeta_visa_descuentos
dataset[foto_mes == 202002, c("ccajeros_propios_descuentos", "mcajeros_propios_descuentos", "ctarjeta_visa_descuentos", "mtarjeta_visa_descuentos", "ctarjeta_master_descuentos", "mtarjeta_master_descuentos") := NA]

# 202009 - ccajeros_propios_descuentos, mcajeros_propios_descuentos, ctarjeta_visa_descuentos
dataset[foto_mes == 202009, c("ccajeros_propios_descuentos", "mcajeros_propios_descuentos", "ctarjeta_visa_descuentos", "mtarjeta_visa_descuentos", "ctarjeta_master_descuentos", "mtarjeta_master_descuentos") := NA]

# 202010 - ccajeros_propios_descuentos, mcajeros_propios_descuentos, ctarjeta_visa_descuentos
dataset[foto_mes == 202010, c("ccajeros_propios_descuentos", "mcajeros_propios_descuentos", "ctarjeta_visa_descuentos", "mtarjeta_visa_descuentos", "ctarjeta_master_descuentos", "mtarjeta_master_descuentos") := NA]

# 202102 - ccajeros_propios_descuentos, mcajeros_propios_descuentos, ctarjeta_visa_descuentos
dataset[foto_mes == 202102, c("ccajeros_propios_descuentos", "mcajeros_propios_descuentos", "ctarjeta_visa_descuentos", "mtarjeta_visa_descuentos", "ctarjeta_master_descuentos", "mtarjeta_master_descuentos") := NA]

# 202105 - ccajas_depositos
dataset[foto_mes == 202105, c("ccajas_depositos") := NA]

### Feature Engineering Historico

A partir del dataset crudo, se genera el campo  clase_ternaria

## Modelado

### Optimizacion de Hipeparámetros

Este script no hace optimización de hiperparámetros
<br> **No** se llama a una Bayesian Optimization, ese paso se saltea.
<br> No hace falta hacer particiones <train, validate, test>,  tampoco se hace un k-fold cross validation

## Produccion

Las decisiones que se toman para la construccion del modelo final son:
* Los positvos son  POS={"BAJA+1", "BAJA+2"}, esta es una meticulosa decisión.
* Se entrena en los cuatro meses  {202101, 202102, 202103, 202104}, esta es una meticulosa decisión.
* Se hace un muy agresivo undersampling del **0.10** = 10% de la clase mayoritaria (los "CONTINUA" )
* Obviamente los datos donde se aplica el modelo es el mes  {202106}
* Por experimentos en meses anteriores, se decide cortar en los 11000 registros con mayor probabildiad de POS={"BAJA+1", "BAJA+2"}, , esta es una *enorme* decisión.
* No se optmizan los hiperparámetros de LightGBM, sino que se llama a la libreria *zLightGBM*
* Para *zLightGBM*  se crean **50** canaritos, esta es una decisión enorme !

## CARGA DE DATOS

In [19]:
nrow(dataset)
ncol(dataset)

[1] 4883051

[1] 155

In [20]:
library(dplyr)

cols_a_eliminar <- c(
  "mprestamos_personales",
  "cprestamos_personales",
    "mrentabilidad_annual"
)

dataset <- dataset %>% 
  select(-all_of(cols_a_eliminar))


In [21]:
# Feature Engineering Historico
# Creacion de LAGs
setorder(dataset, numero_de_cliente, foto_mes)

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
  colnames(dataset),
  c("numero_de_cliente", "foto_mes", "clase_ternaria")
))

# -------------------------
# LAG 1
# -------------------------
dataset[,
  paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# -------------------------
# LAG 2
# -------------------------
dataset[,
  paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# -------------------------
# LAG 3
# -------------------------
dataset[,
  paste0(cols_lagueables, "_lag3") := shift(.SD, 3, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# -------------------------
# LAG 4   (NUEVO)
# -------------------------
dataset[,
  paste0(cols_lagueables, "_lag4") := shift(.SD, 4, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# -------------------------
# DELTAS  (1 a 4)
# -------------------------
for (vcol in cols_lagueables)
{
  # delta1
  dataset[, paste0(vcol, "_delta1") :=
            get(vcol) - get(paste0(vcol, "_lag1"))]

  # delta2
  dataset[, paste0(vcol, "_delta2") :=
            get(vcol) - get(paste0(vcol, "_lag2"))]

  # delta3
  dataset[, paste0(vcol, "_delta3") :=
            get(vcol) - get(paste0(vcol, "_lag3"))]

  # delta4   (NUEVO)
  dataset[, paste0(vcol, "_delta4") :=
            get(vcol) - get(paste0(vcol, "_lag4"))]
}

Sys.time()


[1] "2025-12-03 23:14:46 UTC"

In [22]:
nrow(dataset)
ncol(dataset)

[1] 4883051

[1] 1344

In [23]:
unique(dataset$clase_ternaria)

[1] "CONTINUA" NA         "BAJA+2"   "BAJA+1"

## TRAIN

In [24]:
# training y future
Sys.time()


# Definicion de hiperparametros
PARAM <- list()
PARAM$experimento <- "E2005"
#PARAM$semilla_primigenia <- 100169
PARAM$semilla_canarito <- 1021
# 777781, 777787, 777817, 777839, 777857,100169, 100183, 100189, 100193, 100213

PARAM$meses <- c(201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908, 201909, 201910, 201911, 201912,
                 202001, 202002, 202003, 202004, 202005,202007,202008, 202009, 202010, 202011, 202012, 202101, 202102, 202103,202104,202105,202106,202107)

PARAM$undersampling <- 0.05

PARAM$future <- c(202109)

Sys.time()

[1] "2025-12-03 23:14:46 UTC"

[1] "2025-12-03 23:14:46 UTC"

### Final Training Strategy

Se entrena en los cuatro meses {202101, 202102, 202103, 202104}
<br>Se hace un muy agresivo undersampling del 10% de la clase mayoritaria (los "CONTINUA" )

In [25]:

# training y future
Sys.time()

# se filtran los meses donde se entrena el modelo final
dataset_train_final <- dataset[foto_mes %in% PARAM$meses]
PARAM$semilla_primigenia <- 777787
# Undersampling, van todos los "BAJA+1" y "BAJA+2" y solo algunos "CONTINIA"

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset_train_final[, azar := runif(nrow(dataset_train_final))]
dataset_train_final[, training := 0L]

dataset_train_final[
  (azar <= PARAM$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

dataset_train_final[, azar:= NULL] # elimino la columna azar

[1] "2025-12-03 23:14:46 UTC"

In [26]:
dataset_train_final[,
                    clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

In [27]:
# utilizo  zLightGBM  la nueva libreria
if( !require("zlightgbm") ) install.packages("https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/zlightgbm_4.6.0.99.tar.gz", repos= NULL, type= "source")
require("zlightgbm")
Sys.time()

Loading required package: zlightgbm



[1] "2025-12-03 23:14:51 UTC"

In [28]:
# canaritos
PARAM$qcanaritos <- 5

cols0 <- copy(colnames(dataset_train_final))
filas <- nrow(dataset_train_final)

for( i in seq(PARAM$qcanaritos) ){
  dataset_train_final[, paste0("canarito_",i) := runif( filas) ]
}

# las columnas canaritos mandatoriamente van al comienzo del dataset
cols_canaritos <- copy( setdiff( colnames(dataset_train_final), cols0 ) )
setcolorder( dataset_train_final, c( cols_canaritos, cols0 ) )

Sys.time()

[1] "2025-12-03 23:14:52 UTC"

In [29]:
campos_buenos <- setdiff(
  colnames(dataset_train_final),
  c("clase_ternaria", "clase01", "training")
)

In [30]:
# dejo los datos en el formato que necesita LightGBM

dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train_final[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train_final[training == 1L, clase01],
  free_raw_data= FALSE
)

cat("filas", nrow(dtrain_final), "columnas", ncol(dtrain_final), "\n")
Sys.time()

filas 257919 columnas 1348 


[1] "2025-12-03 23:14:58 UTC"

In [31]:
ncol(dtrain_final)
nrow(dtrain_final)

[1] 1348

[1] 257919

### Target Engineering

In [32]:
#Sys.time()


PARAM$lgbm <-  list(
  boosting= "gbdt",
  objective= "binary",
  metric= "custom",
  first_metric_only= FALSE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  force_row_wise= TRUE,
  lambda_l1= 0.8,
  lambda_l2= 0.8,
  verbosity= -100,
  
  seed= PARAM$semilla_primigenia,
  
  max_bin= 31L,
  min_data_in_leaf= 20L,  #este ya es el valor default de LightGBM
  
  num_iterations= 9999L, # dejo libre la cantidad de arboles, zLightGBM se detiene solo
  num_leaves= 999L, # dejo libre la cantidad de hojas, zLightGBM sabe cuando no hacer un split
  learning_rate= 1.0,  # se lo deja en 1.0 para que si el score esta por debajo de gradient_bound no se lo escale
  
  feature_fraction= 0.50, # un valor equilibrado, habra que probar alternativas ...
  
  canaritos= PARAM$qcanaritos, # fundamental en zLightGBM, aqui esta el control del overfitting
  gradient_bound= 0.05  # default de zLightGBM
)


In [33]:
modelo_final <- lgb.train(
  data= dtrain_final,
  param= PARAM$lgbm
)
Sys.time()

[1] "2025-12-03 23:34:23 UTC"

In [34]:

# aplico el modelo a los datos sin clase
dfuture <- dataset[foto_mes %in% PARAM$future]

In [35]:
# penosamente, en la versión actual de zLightGBM  los campos canaritos
#  aunque no se utilizan para nada, también deben estar en el dataset donde se hace el predict()
filas <- nrow(dfuture)

for( i in seq(PARAM$qcanaritos) ){
  dfuture[, paste0("canarito_",i) := runif( filas) ]
}

prediccion <- predict(
  modelo_final,
  data.matrix(dfuture[, campos_buenos, with= FALSE]),
)

## PREDICCION FINAL

SE TOMA LA DECISION DE HACER 11 MIL ENVIOS

In [36]:
tb_prediccion <- dfuture[, list(numero_de_cliente, foto_mes)]
tb_prediccion[, prob := prediccion ]

In [37]:
fwrite(tb_prediccion, "tb_prediccion5FINAL.csv")

In [38]:
setorder(tb_prediccion, -prob)

envios <- 11000
tb_prediccion[, Predicted := 0L] # seteo inicial a 0
tb_prediccion[1:envios, Predicted := 1L] # marco los primeros

In [39]:
# Filtrar solo los clientes que van a recibir estímulo
tb_submit <- tb_prediccion[Predicted == 1, .(numero_de_cliente)]

# Guardar archivo SIN encabezado, en CSV
fwrite(
  tb_submit,
  file = "submit_envio5F.csv",
  col.names = FALSE,
  sep = ","
)